## Main Notebook for the task

### Step 1 Compute OFI metrics 
1. Derive multi-level OFI metrics (up to 5 levels) for each stock in the dataset.
2. Integrate these multi-level OFIs into a single metric using Principal Component Analysis (PCA) or another dimensionality reduction method.

#### Data-preprocessing
I followed the instructions and retrieved data from AAPL, AMGN, TSLA, JPM, XOM from period 12/07/2024 to 01/07/2025 (one month).  

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import databento as db

In [3]:
data_directory=[]

data_directory_AAPL=f"/Users/mic/Desktop/Github/Cross-Impact-Analysis-of-Order-Flow-Imbalance-OFI-/data/XNAS-20250107-APPL"
data_directory_AMGN=f"/Users/mic/Desktop/Github/Cross-Impact-Analysis-of-Order-Flow-Imbalance-OFI-/data/XNAS-20250107-AMGN"
data_directory_TSLA=f"/Users/mic/Desktop/Github/Cross-Impact-Analysis-of-Order-Flow-Imbalance-OFI-/data/XNAS-20250107-TSLA"
data_directory_JPM=f"/Users/mic/Desktop/Github/Cross-Impact-Analysis-of-Order-Flow-Imbalance-OFI-/data/XNAS-20250107-JPM"
data_directory_XOM=f"/Users/mic/Desktop/Github/Cross-Impact-Analysis-of-Order-Flow-Imbalance-OFI-/data/XNAS-20250107-XOM"

data_directory.append(data_directory_AAPL)
data_directory.append(data_directory_AMGN)
data_directory.append(data_directory_TSLA)
data_directory.append(data_directory_JPM)
data_directory.append(data_directory_XOM)

data_directory

['/Users/mic/Desktop/Github/Cross-Impact-Analysis-of-Order-Flow-Imbalance-OFI-/data/XNAS-20250107-APPL',
 '/Users/mic/Desktop/Github/Cross-Impact-Analysis-of-Order-Flow-Imbalance-OFI-/data/XNAS-20250107-AMGN',
 '/Users/mic/Desktop/Github/Cross-Impact-Analysis-of-Order-Flow-Imbalance-OFI-/data/XNAS-20250107-TSLA',
 '/Users/mic/Desktop/Github/Cross-Impact-Analysis-of-Order-Flow-Imbalance-OFI-/data/XNAS-20250107-JPM',
 '/Users/mic/Desktop/Github/Cross-Impact-Analysis-of-Order-Flow-Imbalance-OFI-/data/XNAS-20250107-XOM']

Making the downloads into dataframe. There are .dbn.zst files for days of MBP-10 order books, we transform it into pandas dataframe and concatenate data for days into a big dataset for each of the five companies. 

In [4]:
combined_dataframes=[]
dataframes=[]
for data_directory_company in data_directory:
    for filename in os.listdir(data_directory_company):
        if filename.endswith(".dbn.zst"):
            file_path=os.path.join(data_directory_company, filename)
            file_df=db.DBNStore.from_file(file_path).to_df()
            dataframes.append(file_df)
    combined_dataframes.append(pd.concat(dataframes, ignore_index=True))
    dataframes=[]

In [7]:
len(combined_dataframes)

5

In [8]:
combined_df_AAPL=combined_dataframes[0]
combined_df_AMGN=combined_dataframes[1]
combined_df_TSLA=combined_dataframes[2]
combined_df_JPM=combined_dataframes[3]
combined_df_XOM=combined_dataframes[4]


In [14]:
combined_df_AAPL.shape

(25602477, 78)

In [9]:
%store combined_df_AAPL
%store combined_df_AMGN
%store combined_df_TSLA
%store combined_df_JPM
%store combined_df_XOM

Stored 'combined_df_AAPL' (DataFrame)
Stored 'combined_df_AMGN' (DataFrame)
Stored 'combined_df_TSLA' (DataFrame)
Stored 'combined_df_JPM' (DataFrame)
Stored 'combined_df_XOM' (DataFrame)


In [10]:
combined_df_AAPL.columns

Index(['ts_event', 'rtype', 'publisher_id', 'instrument_id', 'action', 'side',
       'depth', 'price', 'size', 'flags', 'ts_in_delta', 'sequence',
       'bid_px_00', 'ask_px_00', 'bid_sz_00', 'ask_sz_00', 'bid_ct_00',
       'ask_ct_00', 'bid_px_01', 'ask_px_01', 'bid_sz_01', 'ask_sz_01',
       'bid_ct_01', 'ask_ct_01', 'bid_px_02', 'ask_px_02', 'bid_sz_02',
       'ask_sz_02', 'bid_ct_02', 'ask_ct_02', 'bid_px_03', 'ask_px_03',
       'bid_sz_03', 'ask_sz_03', 'bid_ct_03', 'ask_ct_03', 'bid_px_04',
       'ask_px_04', 'bid_sz_04', 'ask_sz_04', 'bid_ct_04', 'ask_ct_04',
       'bid_px_05', 'ask_px_05', 'bid_sz_05', 'ask_sz_05', 'bid_ct_05',
       'ask_ct_05', 'bid_px_06', 'ask_px_06', 'bid_sz_06', 'ask_sz_06',
       'bid_ct_06', 'ask_ct_06', 'bid_px_07', 'ask_px_07', 'bid_sz_07',
       'ask_sz_07', 'bid_ct_07', 'ask_ct_07', 'bid_px_08', 'ask_px_08',
       'bid_sz_08', 'ask_sz_08', 'bid_ct_08', 'ask_ct_08', 'bid_px_09',
       'ask_px_09', 'bid_sz_09', 'ask_sz_09', 'bid_ct_09', '

Notice that the columns bid_px_N denote the bid price at level N (top level then N=00), bid_sz_N denote the bid size at level N. Similarily for ask levels. 

We implement the best level OFI algorithm as described in the paper

In [11]:
def compute_best_level_ofi(df):
    """
    Compute best-level OFI (Order Flow Imbalance) 
    input df need to have the following columns
      - 'bid_px_01', 'bid_sz_01' (best bid price, size)
      - 'ask_px_01', 'ask_sz_01' (best ask price, size)
    
    The OF formula (based on the essay) for level 1 and for each row t compares row t to t-1 (in essay denoted as update n comparing to update n-1):
      OF_bid(t) = 
          bid_sz_01(t),                 if bid_px_01(t) > bid_px_01(t-1)
         -bid_sz_01(t),              if bid_px_01(t) < bid_px_01(t-1)
          [bid_sz_01(t) - bid_sz_01(t-1)], otherwise
      OF_ask(t) is computed similarly, then
      best_OFI(t) = OF_bid(t) - OF_ask(t).
    """
    
    # Shift columns to compare current row with the previous row
    df['prev_bid_px'] = df['bid_px_01'].shift()
    df['prev_bid_sz'] = df['bid_sz_01'].shift()
    df['prev_ask_px'] = df['ask_px_01'].shift()
    df['prev_ask_sz'] = df['ask_sz_01'].shift()
    
    def row_ofi(row):
        # --- Bid side ---
        if pd.isnull(row['prev_bid_px']):
            # First row (no previous data)
            of_bid = 0.0
        else:
            if row['bid_px_01'] > row['prev_bid_px']:
                of_bid = row['bid_sz_01']
            elif row['bid_px_01'] < row['prev_bid_px']:
                of_bid = -row['bid_sz_01']
            else:
                of_bid = row['bid_sz_01'] - row['prev_bid_sz']
                
        # --- Ask side ---
        if pd.isnull(row['prev_ask_px']):
            # First row (no previous data)
            of_ask = 0.0
        else:
            if row['ask_px_01'] < row['prev_ask_px']:
                of_ask = row['ask_sz_01']
            elif row['ask_px_01'] > row['prev_ask_px']:
                of_ask = -row['ask_sz_01']
            else:
                of_ask = row['ask_sz_01'] - row['prev_ask_sz']
        
        return of_bid - of_ask
    
    # Apply the row_ofi function to each row
    df['best_level_ofi']=df.apply(row_ofi, axis=1)
    return df['best_level_ofi']

In [12]:
best_level_ofi_AAPL = compute_best_level_ofi(combined_df_AAPL)



In [13]:
best_level_ofi_AAPL.shape

(25602477,)

In [ ]:
best_level_ofi_AMGN = compute_best_level_ofi(combined_df_AMGN)
best_level_ofi_TSLA = compute_best_level_ofi(combined_df_TSLA)
best_level_ofi_JPM = compute_best_level_ofi(combined_df_JPM)
best_level_ofi_XOM = compute_best_level_ofi(combined_df_XOM)

In [ ]:
def compute_multi_level_ofi(df, levels=5):
    """
    Compute multi-level OFI across 'levels' levels of the order book.
    We assume your DataFrame has columns like:
      bid_px_0X, bid_sz_0X,
      ask_px_0X, ask_sz_0X
    for X in 0..(levels-1).

    For each level, we use the same calculation method
    Parameters:
        df (pd.DataFrame): Data with order-book columns
        levels (int): How many levels to include (e.g., 5 or 10)
    
    Returns:
        pd.DataFrame: The original DataFrame with extra columns:
                      ofi_lvl_0, ofi_lvl_1, ..., and multi_ofi.
    """
    # For each level, create columns for the previous row's px and sz
    # Then compute an OFI for that level
    for lvl in range(2, 6): #level 2-5
        px_bid_col = f'bid_px_0{lvl}'
        sz_bid_col = f'bid_sz_0{lvl}'
        px_ask_col = f'ask_px_0{lvl}'
        sz_ask_col = f'ask_sz_0{lvl}'
        
        # Shift columns to compare current row with the previous row
        df[f'prev_bid_px_{lvl}'] = df[px_bid_col].shift()
        df[f'prev_bid_sz_{lvl}'] = df[sz_bid_col].shift()
        df[f'prev_ask_px_{lvl}'] = df[px_ask_col].shift()
        df[f'prev_ask_sz_{lvl}'] = df[sz_ask_col].shift()
        
        def row_ofi(row):
            # --- Bid side ---
            if pd.isnull(row[f'prev_bid_px_{lvl}']):
                of_bid = 0.0
            else:
                if row[px_bid_col] > row[f'prev_bid_px_{lvl}']:
                    of_bid = row[sz_bid_col]
                elif row[px_bid_col] < row[f'prev_bid_px_{lvl}']:
                    of_bid = -row[f'prev_bid_sz_{lvl}']
                else:
                    of_bid = row[sz_bid_col] - row[f'prev_bid_sz_{lvl}']
            
            # --- Ask side ---
            if pd.isnull(row[f'prev_ask_px_{lvl}']):
                of_ask = 0.0
            else:
                if row[px_ask_col] < row[f'prev_ask_px_{lvl}']:
                    of_ask = row[sz_ask_col]
                elif row[px_ask_col] > row[f'prev_ask_px_{lvl}']:
                    of_ask = -row[f'prev_ask_sz_{lvl}']
                else:
                    of_ask = row[sz_ask_col] - row[f'prev_ask_sz_{lvl}']
            
            return of_bid - of_ask
        
        # Compute level-specific OFI for level 2-5
        df[f'ofi_lvl_{lvl}'] = df.apply(row_ofi, axis=1)
    
    # retrieve each level OFI as a column and get all levels, should be a x*4 matrix, where 4 columns stands for OFI level 2 to OFI level 5
    ofi_cols = [f'ofi_lvl_{lvl}' for lvl in range(2,6)]
    
    return df[ofi_cols]


Once we have the OFI level 1-5 for each 5 stocks, we can then perform PCA and look for the dominant pinciple component and use it to calculate the ""Integrated OFI""